In [10]:
import tkinter as tk
import pandas as pd

# Initialize the main application window
root = tk.Tk()
root.title("Grade Calculator")
#root.geometry('800x600')
#root.minsize(800,600)

# Frames for layout organization
top_frame = tk.LabelFrame(root, text="Student Information")
top_frame.grid(row=0, column=0, columnspan=4, padx=10, pady=10, sticky="ew")

assessment_frame = tk.LabelFrame(root, text="Assessments")
assessment_frame.grid(row=1, column=0, columnspan=4, padx=10, pady=10, sticky="nsew")

bottom_frame = tk.Frame(root)
bottom_frame.grid(row=2, column=0, columnspan=4, padx=10, pady=10, sticky="ew")

# Initialize a DataFrame to store the data
df_columns = ['Subject', 'Student ID', 'Student Name', 'Final Grade']
df = pd.DataFrame(columns=df_columns)

# Lists to hold the widgets dynamically
assessment_entries = []
weights = []
grades = []

def calculate_final_grade():
    try:
        # Calculate the weighted average for the final grade
        total_weight = sum(float(weight.get()) for weight in weights if weight.get())
        weighted_grades = sum(float(grade.get()) * float(weight.get()) for grade, weight in zip(grades, weights) if grade.get() and weight.get())
        final_grade = weighted_grades / total_weight if total_weight != 0 else 0
        final_grade_entry.delete(0, tk.END)
        final_grade_entry.insert(0, str(final_grade))
        
        # Construct data dictionary dynamically based on the number of assessments
        data = {
            'Subject': subject_entry.get(),
            'Student ID': student_id_entry.get(),
            'Student Name': student_name_entry.get(),
            'Final Grade': final_grade_entry.get()
        }
        for i, (assessment_entry, weight, grade) in enumerate(zip(assessment_entries, weights, grades)):
            data[f'Assessment {i+1}'] = assessment_entry.get()
            data[f'Weight {i+1}'] = weight.get()
            data[f'Grade {i+1}'] = grade.get()

        # Append the data to the DataFrame
        global df
        df = pd.concat([df, pd.DataFrame([data])], ignore_index=True)
        
        # Save the DataFrame to a CSV file
        df.to_csv('~/Documents/calificaMe/output/grades.csv', index=False)
        
        # Notify the user that the data was saved successfully
        print("Data saved successfully.")
        
        # Clear the Student ID, Student Name, and Grade fields
        student_id_entry.delete(0, tk.END)
        student_name_entry.delete(0, tk.END)
        for grade_entry in grades:
            grade_entry.delete(0, tk.END)
        
    except Exception as e:
        print("An error occurred:", e)
        # You might want to show a dialog box or another form of error notification here

# GUI Layout for the entries in the top frame
tk.Label(top_frame, text="Subject:").grid(row=0, column=0, sticky="e")
subject_entry = tk.Entry(top_frame)
subject_entry.grid(row=0, column=1, sticky="ew")

tk.Label(top_frame, text="Student ID:").grid(row=1, column=0, sticky="e")
student_id_entry = tk.Entry(top_frame)
student_id_entry.grid(row=1, column=1, sticky="ew")

tk.Label(top_frame, text="Student Name:").grid(row=1, column=2, sticky="e")
student_name_entry = tk.Entry(top_frame)
student_name_entry.grid(row=1, column=3, sticky="ew")

# Calculate button and Final Grade entry in the bottom frame
calculate_button = tk.Button(bottom_frame, text="Calculate", command=calculate_final_grade)
calculate_button.pack(side=tk.LEFT, padx=5)

final_grade_entry = tk.Entry(bottom_frame)
final_grade_entry.pack(side=tk.LEFT, padx=5)

# Column titles in the assessment_frame
tk.Label(assessment_frame, text="Assessment").grid(row=0, column=0, sticky="ew")
tk.Label(assessment_frame, text="Weight").grid(row=0, column=1, sticky="ew")
tk.Label(assessment_frame, text="Grade").grid(row=0, column=2, sticky="ew")

# Functions for adding and removing assessments
def add_assessment():
    # Create new set of entries for assessment, weight, and grade
    assessment_entry = tk.Entry(assessment_frame)
    weight_entry = tk.Entry(assessment_frame)
    grade_entry = tk.Entry(assessment_frame)
    
    # Place them in the grid
    assessment_entry.grid(row=len(assessment_entries)+1, column=0, sticky="ew")
    weight_entry.grid(row=len(weights)+1, column=1, sticky="ew")
    grade_entry.grid(row=len(grades)+1, column=2, sticky="ew")
    
    # Add them to the lists
    assessment_entries.append(assessment_entry)
    weights.append(weight_entry)
    grades.append(grade_entry)

    # Update DataFrame columns
    update_df_columns()

def remove_assessment():
    if assessment_entries:
        # Remove the last set of entries
        assessment_entry = assessment_entries.pop()
        weight_entry = weights.pop()
        grade_entry = grades.pop()

        # Destroy the widgets
        assessment_entry.destroy()
        weight_entry.destroy()
        grade_entry.destroy()

        # Update DataFrame columns
        update_df_columns()

def update_df_columns():
    # Adjust the DataFrame columns based on the number of assessments
    num_assessments = len(assessment_entries)
    new_columns = df_columns[:3] + [f'Assessment {i+1}' for i in range(num_assessments)] + \
                  [f'Weight {i+1}' for i in range(num_assessments)] + \
                  [f'Grade {i+1}' for i in range(num_assessments)] + \
                  ['Final Grade']
    global df
    df = df.reindex(columns=new_columns)

# Add and Remove Assessment buttons in the top frame
add_assessment_button = tk.Button(top_frame, text="Add Assessment", command=add_assessment)
add_assessment_button.grid(row=2, column=0, columnspan=2, sticky="ew")

remove_assessment_button = tk.Button(top_frame, text="Remove Assessment", command=remove_assessment)
remove_assessment_button.grid(row=2, column=2, columnspan=2, sticky="ew")

# Start the application
root.mainloop()
